In [ ]:
!pip install -q gradio

import gradio as gr
import zipfile
import os
import email
from email import policy
from email.parser import BytesParser
import traceback

In [ ]:
def parse_email(file_path):
    """
    Parse a .eml file path.
    Return dict: subject, sender, body, extracted tasks.
    """
    with open(file_path, 'rb') as fp:
        msg = BytesParser(policy=policy.default).parse(fp)

    subject = msg['subject'] or "No Subject"
    sender = msg['from'] or "Unknown Sender"
    body = msg.get_body(preferencelist=('plain'))
    body_text = body.get_content() if body else "No content"

    # Dummy NLP: tasks = lines starting with "-"
    tasks = [line.strip() for line in body_text.splitlines() if line.strip().startswith("-")]

    return {
        "filename": os.path.basename(file_path),
        "subject": subject,
        "from": sender,
        "body": body_text,
        "tasks": tasks
    }

In [ ]:
def create_reply_draft(email_info):
    """
    Given parsed email info, make a reply draft text.
    """
    reply = f"Subject: Re: {email_info['subject']}\n\nHi,\n\n"
    if email_info['tasks']:
        reply += "Here is a summary of tasks:\n"
        for i, task in enumerate(email_info['tasks'], 1):
            reply += f"{i}. {task}\n"
    else:
        reply += "Could you clarify next steps?\n"
    reply += "\nBest regards,"
    return reply

In [ ]:
def create_drafts_zip(drafts, zip_path="/content/email_drafts.zip"):
    """
    drafts: list of (filename, content)
    Write to ZIP and return path.
    """
    with zipfile.ZipFile(zip_path, "w") as z:
        for name, content in drafts:
            z.writestr(name, content)
    return zip_path

In [ ]:
def process_emails(file_paths):
    try:
        results = []
        drafts = []

        for path in file_paths:
            info = parse_email(path)
            results.append(info)

            draft_content = create_reply_draft(info)
            draft_name = f"{info['filename']}_reply.txt"
            drafts.append( (draft_name, draft_content) )

        # Make ZIP
        zip_path = create_drafts_zip(drafts)

        # Pretty HTML
        html = "<h2>📧 Processed Emails:</h2>"
        for info in results:
            html += f"<h3>{info['filename']}</h3>"
            html += f"<b>Subject:</b> {info['subject']}<br>"
            html += f"<b>From:</b> {info['from']}<br>"
            html += f"<b>Tasks:</b> {len(info['tasks'])}<ul>"
            for t in info['tasks']:
                html += f"<li>{t}</li>"
            html += "</ul>"

        return html, zip_path

    except Exception as e:
        print("\n🔴 FULL TRACEBACK:")
        traceback.print_exc()
        raise

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📬 **DocAgent Email Processor**\nUpload .eml files → See tasks → Download drafts ZIP")

    files_input = gr.Files(label="Upload .eml files")
    process_btn = gr.Button("📑 Process Emails & Make ZIP")

    html_out = gr.HTML()
    zip_out = gr.File(label="Download All Drafts (ZIP)")

    process_btn.click(
        process_emails,
        inputs=files_input,
        outputs=[html_out, zip_out]
    )

demo.launch(share=True, debug=True)